In [1]:
# Initial setup cell
!pip3 install -r ../../requirements.txt

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)


  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 6.2 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.8 MB/s eta 0:00:00
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
Using cached gast-0.5.4-py3-none-any.whl (19 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 32.9 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 31.8 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Import dependencies
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

### Data Setup
Initializes our train and test set and stores the ground truth of each image. We use Torch libraries to handle this for us.

In [2]:
# Import data setup dependencies
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from sklearn.model_selection import KFold

The code below extracts images from our dataset, resizes each into a fourth their original size (768 -> 192), and converts them into Torch tensors. The ImageFolder class allows us to lazyload our images to preserve our computational power.

In [3]:
# Path to our lung_image_sets
data_dir = "../../lung_colon_image_set/lung_image_sets"

# Set the resize size of the images
resized_size = 80

# Convert images into Tensors
tensor_data = transforms.Compose([
  transforms.Resize((resized_size, resized_size)),   # Cut image into a fourth of original size
  transforms.ToTensor()
])

# Load the dataset using ImageFolder
data = ImageFolder(root=data_dir, transform=tensor_data)

# Split the dataset into train and test sets
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train, test = torch.utils.data.random_split(data, [train_size, test_size])

# Create data loaders for training and testing
load_train = DataLoader(train, batch_size=32, shuffle=True)
load_test = DataLoader(test, batch_size=32, shuffle=False)

### DGW-Net: A basic CNN image classifier

DGW-Net (abbreviated from Dino-Garcia-Wang Net) is a simple CNN architecture that follows the structure below:
- Input -> CONV (3x3, 64 filters) -> ReLU -> MaxPool -> CONV (3x3, 64 filters) -> ReLU -> MaxPool -> FC

This model architecture follows from our simple CNN implementation for CIFAR-10 on Assignment 2. We decided on 64 filters as empirically this number has proven to be successful in models like ResNet. The implementation follows below:

In [4]:
# Define our DGW-Net Architecture
class DGWNet(nn.Module):
    def __init__(self):
        super(DGWNet, self).__init__()
        
        # First convolutional block: 64 filters used. (L, L, 3) -> (L//2, L//2, 64)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second convolutional block: 64 filters used. (L//2, L//2, 64) -> (L//4, L//4, 64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # FC Layer -> 128 Features
        self.fc = nn.Linear(resized_size // 4 * resized_size // 4 * 64, 128)
        
    def forward(self, x):
        # First convolutional block
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        # Second convolutional block
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        # Flatten the output from the convolutional blocks
        x = x.view(x.size(0), -1)
        
        # Fully connected layer for classification
        x = self.fc(x)
        
        return x

### Hyperparameters and Constants
Here we define our hyperparameters and constants that will stay constant throughout all implementations of DGW-Net.

In [ ]:
# Define hyperparameters
learning_rate = 5e-4
momentum = 0.9

# Define number of epochs
num_epochs = 1

# Define number of folds
k_folds = 5

# Create a KFold object with 5 splits
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=231)
folds = kfold.split(data)

### DGW-Net + SVM
For our first situation, we will use SVM to do classification on our extracted features.

In [5]:
class DGWSVM(nn.Module):
  def __init__(self):
      super(DGWSVM, self).__init__()
      
      # Base DGWNet feature extractor -> 128 features
      self.extract_features = DGWNet()
      
      # SVM layer
      self.svm = nn.Linear(128, 3) 
  
  def forward(self, x):
        # Pass the input through the base CNN
        x = self.extract_features(x)
        
        # SVM layer
        x = self.svm(x)
        
        return x

In [6]:
# Define instance of our model
svm_model = DGWSVM()

# Define our loss function and optimizer
loss_function = nn.CrossEntropyLoss()
svm_optimizer = optim.SGD(svm_model.parameters(), lr=learning_rate, momentum=momentum)

# Check current device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Now, let's train our DGW-SVM model!

In [ ]:
# Iterate over folds in training
for fold, (train_indices, val_indices) in enumerate(folds, 1):
    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(data, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(data, batch_size=32, sampler=val_sampler)

    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for images, labels in load_train:
            # Move the input data to the device (CPU or GPU)
            images = images.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            svm_optimizer.zero_grad()
            
            # Forward pass
            outputs = svm_model(images)
            loss = loss_function(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            svm_optimizer.step()
            
            # Update running loss
            running_loss += loss.item()
        
        # Print the average loss for the epoch
        epoch_loss = running_loss / len(load_train)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

### DGW-Net + Softmax
For our second situation, we will use Softmax to do classification on our extracted features.

In [ ]:
class DGWSoftmax(nn.Module):
  def __init__(self):
      super(DGWSoftmax, self).__init__()
      
      # Base DGWNet feature extractor -> 128 features
      self.extract_features = DGWNet()
      
      # Pre-softmax FC layer
      self.fc = nn.Linear(128, 3) 
  
  def forward(self, x):
        # Pass the input through the base CNN
        x = self.extract_features(x)
        
        # Softmax layer
        x = self.fc(x)

        # Apply softmax activation
        x = F.softmax(x, dim=1)
        
        return x

In [ ]:
# Define instance of our model
softmax_model = DGWSoftmax()

# Define our optimizer (reuse same loss function from before)
softmax_optimizer = optim.SGD(softmax_model.parameters(), lr=learning_rate, momentum=momentum)

Now, let's train our DGW-Softmax model:

In [ ]:
# Iterate over folds in training
for fold, (train_indices, val_indices) in enumerate(folds, 1):
    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(data, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(data, batch_size=32, sampler=val_sampler)

    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for images, labels in load_train:
            # Move the input data to the device (CPU or GPU)
            images = images.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            softmax_optimizer.zero_grad()
            
            # Forward pass
            outputs = softmax_model(images)
            loss = loss_function(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            softmax_optimizer.step()
            
            # Update running loss
            running_loss += loss.item()
        
        # Print the average loss for the epoch
        epoch_loss = running_loss / len(load_train)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")